In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import shutil
import sys
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import tensorflow as tf
import bottleneck as bn
import keras
import math

In [3]:
import argparse
import pickle
import time
import numpy as np

In [4]:
cd /content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master/

/content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master


In [5]:
train_data = pickle.load(open('../datasets/' + 'diginetica' + '/train.txt', 'rb'))

In [12]:

def build_graph(train_data):
    graph = nx.DiGraph()
    for seq in train_data:
        for i in range(len(seq) - 1):
            if graph.get_edge_data(seq[i], seq[i + 1]) is None:
                weight = 1
            else:
                weight = graph.get_edge_data(seq[i], seq[i + 1])['weight'] + 1
            graph.add_edge(seq[i], seq[i + 1], weight=weight)
    for node in graph.nodes:
        sum = 0
        for j, i in graph.in_edges(node):
            sum += graph.get_edge_data(j, i)['weight']
        if sum != 0:
            for j, i in graph.in_edges(i):
                graph.add_edge(j, i, weight=graph.get_edge_data(j, i)['weight'] / sum)
    return graph


def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        '''
        print("##### in get_slice, inputs")
        print(inputs)
        print(len(inputs),len(inputs[0]),len(inputs[1]))
        print("##### in get_slice, mask")
        print(mask)
        print("##### in get_slice, targets")
        print(targets)
        '''
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input))) 
        max_n_node = np.max(n_node) # 해당 slices 중 한 sequence 안에서 가질 수 있는 아이템의 종류의 최댓값
        # 0 패딩 후에 개수를 따지는 것이기 때문에 id 개수 +1 임.
        
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node)) # 균일한 크기로 생성.
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                print("node")
                print(node)
                print('u_input[i]')
                print(u_input[i])
                print("np.where(node == u_input[i])")
                print(np.where(node == u_input[i]))
                print("np.where(node == u_input[i + 1])")
                print(np.where(node == u_input[i + 1]))
                break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1 # i 시점 i+1 이점에  인접해 있으면 1을 기록.

            break
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets


In [13]:
train_data, valid_data = split_validation(train_data, 0.1)
test_data = valid_data

In [14]:
train_data1 = Data(train_data, shuffle=False)
test_data1 = Data(test_data, shuffle=False)

In [15]:
slices = train_data1.generate_batch(100)


for i, j in zip(slices, np.arange(len(slices))):
    
    #alias_inputs, A, items, mask, targets = train_data1.get_slice(i)
    train_data1.get_slice(i)
    break

node
[    0 15208 38809 40302 42727]
u_input[i]
40302
np.where(node == u_input[i])
(array([3]),)
np.where(node == u_input[i + 1])
(array([4]),)


In [ ]:
class opt:
  def __init__(self,step,hidden_size,batch_size,nonhybrid=None):
    self.step=step
    self.hiddenSize = hidden_size
    #self.n_node = n_node
    self.batchSize = batch_size
    self.nonhybrid = nonhybrid
    self.lr = 0.001
    self.l2 = 1e-5
    self.lr_dc_step = 3
    self.lr_dc = 0.1
    self.epoch = 30
    self.patience = 10
    

In [ ]:
oopt=opt(step = 1,hidden_size=100,batch_size=100)

In [ ]:
n_node = 43098

In [ ]:
class GNN(tf.keras.layers.Layer):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        # self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.stdv = 1/math.sqrt(self.hidden_size)
        self.w_ih = self.add_weight(shape=(self.input_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        AttributeError: module 'keras.api._v2.keras.initializers' has no attribute 'Uniform'
        add_weight doesn't have the attribute named 'kernel_initializer'
        '''
        # self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.w_hh = self.add_weight(shape=(self.hidden_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_ih = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        
        # self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
                
        # self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_iah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_oah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_in = tf.keras.layers.Dense(self.hidden_size, 
                                                    use_bias=True,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        TypeError: ('Unknown keyword argument:', 'bias') - > use_bias=True in Dense
        '''
        # self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

    def Cell(self, A, hidden):
        input_in = tf.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = tf.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]],self.linear_edge_out(hidden)) + self.b_oah
        inputs = tf.concat([input_in, input_out], 2) 
        gi = tf.matmul(inputs, self.w_ih )+ self.b_ih # Dense layer와 같은 역할함.
        # gate_size가 앞서서 3* hidden_size로 정의되어 있었고 아래 chunk에서 split하는 것 같다.
        gh = tf.matmul(hidden, self.w_hh) + self.b_hh
        i_r, i_i, i_n = tf.split(gi,3,axis = 2)
        h_r, h_i, h_n = tf.split(gh,3,axis = 2)
        resetgate = tf.nn.sigmoid(i_r + h_r)
        inputgate = tf.nn.sigmoid(i_i + h_i)
        newgate = tf.nn.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def call(self, A, hidden):
        for i in range(self.step):
            hidden = self.Cell(A, hidden)
        return hidden

In [ ]:

class TAGNN(tf.keras.models.Model):
    def __init__(self, opt, n_node):
        super(TAGNN, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.batch_size = opt.batchSize
        self.nonhybrid = opt.nonhybrid
        # self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.embedding = tf.keras.layers.Embedding(input_dim = self.n_node,output_dim=self.hidden_size)
        self.stdv = 1/math.sqrt(self.hidden_size)
        
        self.gnn = GNN(self.hidden_size, step=opt.step)

        # self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_one = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_three = tf.keras.layers.Dense(1 , use_bias=False,
                                                  kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                  )
        # self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        
        self.linear_transform = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                      kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                      bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_t = nn.Linear(self.hidden_size, self.hidden_size, bias=False)  #target attention
        self.linear_t = tf.keras.layers.Dense( self.hidden_size, use_bias=False,
                                              kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                              )
    def call(self, inputs, A, mask,alias_inputs):
        if len(self.embedding.weights) != 0:
          norm = tf.norm(self.embedding.weights[0],axis=1) # 왜 여기도 하는지는 모르겠음.
          self.embedding.weights[0]=self.embedding.weights[0] / tf.reshape(norm,[-1,1])
        hidden = self.embedding(inputs) 
        # inputs = items -> batch_size, len_seq_in_batch


        hidden = self.gnn(A, hidden)
        get = lambda j: tf.gather(hidden[j],alias_inputs[j])
        hidden = tf.stack([get(j) for j in range(len(alias_inputs))])

        seq_shape = hidden.shape # 여기 해보아야 알듯 리스트로 차원반환하는 것 같은데
        # seq_hidden = seq_hidden.view(-1, model.hidden_size) # tf.reshape
        hidden = tf.reshape(hidden,[-1,self.hidden_size])
        # norms = torch.norm(seq_hidden, p=2, dim=1)  # tf.norm
        norms = tf.norm(hidden,axis =1)
        # seq_hidden = seq_hidden.div(norms.unsqueeze(-1).expand_as(seq_hidden)) # unsqueeze에 대해 알아보기
        hidden = hidden / tf.reshape(norms,[-1,1])
        # expand_as는 broadcasting으로 해결될 수 있는지 확인해보기.
        # seq_hidden = seq_hidden.view(seq_shape) #  
        
        hidden = tf.reshape(hidden,seq_shape)

        #print(self.embedding.weights)
        ht = tf.gather_nd(hidden ,indices = tf.stack([tf.range(mask.shape[0],dtype='int64') , tf.cast(tf.math.reduce_sum(mask,1),'int64') -1],axis=1))
        # tf.reshape을 이용해야함.
        q1 = tf.reshape(self.linear_one(ht) ,[ht.shape[0], 1, ht.shape[1]])  # batch_size x 1 x latent_size
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        alpha = self.linear_three(tf.nn.sigmoid(q1 + q2))  # (b,s,1)
        
        alpha = tf.nn.softmax(alpha, 1) # B,S,1
        
        
        a = tf.math.reduce_sum(alpha * hidden * tf.cast(tf.reshape(mask, [mask.shape[0], -1, 1]),dtype = 'float32'), 1)  # (b,d)
        if not self.nonhybrid: # 여기 적용되는지 판단할 것
        
            a = self.linear_transform(tf.concat([a, ht], 1)) 
        
        norm = tf.norm(self.embedding.weights[0],axis=1)
        self.embedding.weights[0]=self.embedding.weights[0] / tf.reshape(norm,[-1,1])

        b = self.embedding.weights[0][1:] # 이 차원이 맞긴 함.

        hidden = hidden * tf.cast(tf.reshape(mask,[mask.shape[0], -1, 1]),'float32')  # batch_size x seq_length x latent_size
        

        qt = self.linear_t(hidden)  # batch_size x seq_length x latent_size

        beta = tf.nn.softmax(b @ tf.transpose(qt,[0,2,1]), -1)  # batch_size x n_nodes x seq_length
        target = beta @ hidden  # batch_size x n_nodes x latent_size
        a = tf.reshape(a,[ht.shape[0], 1, ht.shape[1]])  # b,1,d
        a = a + target  # b,n,d # 이거 concat인가 ? 

        scores = tf.math.reduce_sum(a * b, -1)  # b,n
        scores=tf.nn.softmax(scores,-1)
        return scores

# NISER 논문에서 나온 것처럼 처음 모델이 gnn이전에 embedding layer의 weights을 normalize하는 것을 call 안에서 짜려고 했는데, 처음 embedding layer가 호출 되기 이전까지 weight이 존재하지 않아서 다른 방식으로 짜야할 것 같음.

- build를 따로 둔다거나?
- Input을 사용해서 정의하는 것 ?
sequential이나
funtional api에서 하는 것처럼 일반적인 keras
built in을 사용하면 된다 ?


- 일단 미봉책으로 weight이 길이가 0인 경우 pass 한 다음 이후 embedding layer의 weights이 생겨난 후부터 normalize를 적용하도록 했음.



```python
import numpy as np

inputs = keras.Input(shape=(3,))
outputs = ActivityRegularizationLayer()(inputs)
model = keras.Model(inputs, outputs)

# If there is a loss passed in `compile`, the regularization
# losses get added to it
model.compile(optimizer="adam", loss="mse")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))

# It's also possible not to pass any loss in `compile`,
# since the model already has a loss to minimize, via the `add_loss`
# call during the forward pass!
model.compile(optimizer="adam")
model.fit(np.random.random((2, 3)), np.random.random((2, 3)))
```

In [ ]:
model=TAGNN(oopt,n_node)

loss_ftn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = oopt.lr)


@tf.function
def train_ftn(item_input,A_input,mask_input,alias_inputs_,targets_input):
    with tf.GradientTape() as tape :
      scores = model(item_input, A_input, mask_input,alias_inputs_,training =True)
      
      ll = loss_ftn(targets_input-1,scores)

    
    
    gradients =tape.gradient(ll,model.trainable_weights)

    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    return ll

In [ ]:
x = tf.reshape(tf.range(10),[5,-1])

In [ ]:
x

<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]], dtype=int32)>

In [ ]:
y = tf.constant([2,0,5,0,8])

In [ ]:
result=tf.cast(x == y[:,tf.newaxis],tf.int32)

In [ ]:
result

<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[0, 0],
       [0, 0],
       [0, 1],
       [0, 0],
       [1, 0]], dtype=int32)>

In [ ]:
result_hit = tf.math.reduce_sum(result,axis =1)

In [ ]:
result_hit

<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 0, 1, 0, 1], dtype=int32)>

In [ ]:
result_hit = tf.cast(result == 0,tf.int32) # hit

In [ ]:
result_hit

<tf.Tensor: shape=(5, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1],
       [1, 0],
       [1, 1],
       [0, 1]], dtype=int32)>

In [ ]:
index=tf.range(result.shape[-1])+1

In [ ]:
index

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([1, 2], dtype=int32)>

In [ ]:
result_mrr = tf.reduce_sum(result/tf.reshape(index,[-1,index.shape[0]]), axis =-1)

In [ ]:
result_mrr

<tf.Tensor: shape=(5,), dtype=float64, numpy=array([0. , 0. , 0.5, 0. , 1. ])>

In [ ]:
@tf.function
def test_ftn(item_input,A_input,mask_input,alias_inputs_,targets_input):
    scores = model(item_input, A_input, mask_input,alias_inputs_,training =False)
    scores = tf.math.top_k(scores,k=20).indices
    scores = tf.cast(scores,tf.int64)
    targets_input = tf.cast(targets_input,tf.int64)

    #print(scores == targets_input[:,tf.newaxis]-1)
    result=tf.cast(scores == (targets_input[:,tf.newaxis]-1),tf.float32)

    result_hit = tf.math.reduce_sum(result,axis =1)
    
    result_hit = tf.cast(result_hit != 0,tf.float32) # hit

    index=tf.cast(tf.range(result.shape[-1])+1,tf.float32)


    result_mrr = tf.reduce_sum(result/tf.reshape(index,[-1,index.shape[0]]), axis =-1)


    return result_hit, result_mrr

In [ ]:
for epoch in range(oopt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)

    slices = train_data1.generate_batch(model.batch_size)
    loss = 0
    for i, j in zip(slices, np.arange(len(slices))):
        
        alias_inputs, A, items, mask, targets = train_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)
        
        l = train_ftn(items,A,mask,alias_inputs,targets)
        
        loss += l 
        
        if j % int(100) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), l))
        
    print('\tLoss:\t%.3f' % loss)
    hit, mrr = tf.constant([],dtype= tf.float32), tf.constant([],dtype= tf.float32)
    hhit=[]
    mmrr=[]
    slices = test_data1.generate_batch(model.batch_size)
    for i in slices:
        alias_inputs, A, items, mask, targets = test_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)

        h,m=test_ftn(items,A,mask,alias_inputs,targets)

        hit = tf.concat([hit,h],0)
        mrr = tf.concat([mrr,m],0)
        '''
        scores = model(items, A, mask,alias_inputs,training =False)
        scores = tf.math.top_k(scores,k=20).indices
        scores = scores.numpy()
        
            # sub_scores를 tensor에서 numpy로 바꾸고 아래는 그대로 실행하면 됨.
        for score, target, mask in zip(scores, targets, test_data1.mask):
            hhit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
                # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

                # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
            if len(np.where(score == target - 1)[0]) == 0:
                mmrr.append(0)
            else:
                # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
                mmrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
        '''

    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100


    print(f"Recall@20 : {hit} , MRR@20 {mrr}")


-------------------------------------------------------
epoch:  0
[0/6476] Loss: 10.6689
[100/6476] Loss: 10.1723
[200/6476] Loss: 10.2705
[300/6476] Loss: 9.9734
[400/6476] Loss: 9.8514
[500/6476] Loss: 9.8151
[600/6476] Loss: 10.0088
[700/6476] Loss: 10.0381
[800/6476] Loss: 9.8118
[900/6476] Loss: 9.7974
[1000/6476] Loss: 9.8826
[1100/6476] Loss: 9.4438
[1200/6476] Loss: 9.5709
[1300/6476] Loss: 9.0590
[1400/6476] Loss: 8.9560
[1500/6476] Loss: 9.3193
[1600/6476] Loss: 8.7535
[1700/6476] Loss: 8.9450
[1800/6476] Loss: 8.4926
[1900/6476] Loss: 8.9383
[2000/6476] Loss: 8.4592
[2100/6476] Loss: 8.7921
[2200/6476] Loss: 8.4583
[2300/6476] Loss: 8.2696
[2400/6476] Loss: 7.6785
[2500/6476] Loss: 8.0041
[2600/6476] Loss: 7.6961
[2700/6476] Loss: 7.8852
[2800/6476] Loss: 7.4889
[2900/6476] Loss: 7.5765
[3000/6476] Loss: 7.3732
[3100/6476] Loss: 7.2097
[3200/6476] Loss: 7.0322
[3300/6476] Loss: 6.7970
[3400/6476] Loss: 7.1049
[3500/6476] Loss: 6.9861
[3600/6476] Loss: 6.8211
[3700/6476] Loss

KeyboardInterrupt: ignored

In [ ]:
mrr

In [ ]:
np.mean(hhit)

In [ ]:
mmrr

In [ ]:
    scores = tf.math.top_k(scores,k=20).indices
    scores = tf.cast(scores,tf.int64)
    scores = tf.cast(targets,tf.int64)
    
    result=tf.cast(scores == targets[:,tf.newaxis]-1,tf.float32)

In [ ]:
result

In [ ]:
hit=[]
mrr=[]

In [ ]:
#        scores = model(items, A, mask,alias_inputs,training =False)
scores = tf.math.top_k(scores,k=20).indices
scores = scores.numpy()
for score, target, mask in zip(scores, targets, test_data1.mask):
    hit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
        # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

        # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
    if len(np.where(score == target - 1)[0]) == 0:
        mrr.append(0)
    else:
        # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
        mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))

In [ ]:
hit

# 1회 epoch
```
 Recall@20 : 45.50432957593784 , MRR@20 15.308210672538506
```
# 2회 epoch

```
Recall@20 : 51.77839242775932 , MRR@20 17.690032606197565
```

# 3회 epoch
```
Recall@20 : 52.64569752734653 , MRR@20 18.16359554471299
```

Recall@20 : 51.98409940650757 , MRR@20 18.082734233101373